In [1]:
# Dependency installs
!pip install tensorflow --upgrade &> /dev/null
!pip install tqdm &> /dev/null
# DataSet download
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-02-24 12:47:05--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 66.102.1.128, 2a00:1450:400c:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|66.102.1.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  97.4MB/s    in 0.7s    

2020-02-24 12:47:06 (97.4 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as pltprint("Fine Tuning achieved a performance difference of {} percent from the transfer learning model".format(validation_accuracy_fine_tuned-validation_accuracy))
%matplotlib inline
import tensorflow as tf
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from contextlib import redirect_stdout # To save model.summary() to file

# Transfer Learning

In [0]:
# Unpacking the dataset
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(dataset_path, mode='r')
zip_object.extractall("./")
zip_object.close()

# Setting the dataset paths
dataset_path_new = "./cats_and_dogs_filtered/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [0]:
# Helper function to save the model.summary() to filename.txt
def save_model_summary_to_file(filename, model):
    with open(filename, 'w') as f:
        with redirect_stdout(f):
            model.summary()

In [5]:
# Image shape set to constant for further use
IMG_SHAPE = (128, 128, 3)

# Loading a pretrained model
# Using the MobileNetV2
# MobileNetV2 is trained on the imagenet dataset.
# include_top is set to false as we need to use our custom dataset instead of 
#   feeding in imagenet data/dimensions
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, 
                                               include_top=False, 
                                               weights="imagenet")

# base_model.summary() # Saving to file instead
save_model_summary_to_file('BaseModel_MobileNetV2.txt', base_model)

# Freezing the base model
base_model.trainable = False

9412608/9406464 [==============================] - 1s 0us/step


## Defining a custom head for the network

In [6]:
base_model.output # shape = (batch, height, width, channels)

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

4x4x1280 is not suited for the output layer of the custom head. Either just outright flatten it (still too many parameters). The other option is to use Pooling - GlobalAveragePooling. Global pooling takes the pooling from the whole input instead of processing parts of it at a time with a sliding stride window. It reduces the input size significantly.

In [7]:
# Layer that pools across all the filters. Just pass base model's o/p to it.
# Note the new sweet way of giving the input to one layer as (another_layer.output)
# I never knew that
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer # shape = (batch, channels)

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
# Making the prediction layer
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

## Defining our model

In [0]:
# Combine the base model with our model by passing in inputs and outputs
model = tf.keras.models.Model(inputs=base_model.input,
                              outputs=prediction_layer)

# Saving the summary to file
save_model_summary_to_file('combinedModel.txt', model=model)

## Compile the Model

In [0]:
# Optimizer: RMSP: proven to be the best opt for MobileNetv2
#               smaller learning rate since we are using a pretrained network
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Creating Data Generators for Preprocessing

MobileNetV2 Only supports certain input sizes

(96,96), (128,128), (160,160), (192,192), (224,224)

So, we need to resize

from tensorflow.keras.preprocessing.image import ImageDataGenerator

will do the necessary preprocessing steps


General practice is to create two generators - one for training and one for testing dataset

In [0]:
# Normalize
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_validation = ImageDataGenerator(rescale=1/255.)

In [12]:
# Specify where to find the dataset
# Takes the path to a  directory and generates batches of augmented data
train_generator = data_gen_train.flow_from_directory(directory=train_dir,
                                                     target_size=(128,128),
                                                     class_mode='binary',
                                                     batch_size=128)

Found 2000 images belonging to 2 classes.


In [13]:
validation_generator = data_gen_validation.flow_from_directory(directory=validation_dir,
                                                               target_size=(128,128),
                                                               class_mode='binary',
                                                               batch_size=128)

Found 1000 images belonging to 2 classes.


## Training the Transfer Learning Model

In [16]:
# Instead of the usual model.train, use model.fit_generator()
model.fit_generator(generator=train_generator,
                    epochs=10,
                    validation_data=validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/10
16/16 [==============================] - 9s 557ms/step - loss: 0.4534 - accuracy: 0.7945 - val_loss: 0.3588 - val_accuracy: 0.8700
Epoch 2/10
16/16 [==============================] - 9s 566ms/step - loss: 0.4261 - accuracy: 0.8125 - val_loss: 0.3340 - val_accuracy: 0.8770
Epoch 3/10
16/16 [==============================] - 9s 578ms/step - loss: 0.4020 - accuracy: 0.8290 - val_loss: 0.3145 - val_accuracy: 0.8820
Epoch 4/10
16/16 [==============================] - 9s 564ms/step - loss: 0.3806 - accuracy: 0.8480 - val_loss: 0.2977 - val_accuracy: 0.8870
Epoch 5/10
16/16 [==============================] - 9s 561ms/step - loss: 0.3611 - accuracy: 0.8610 - val_loss: 0.2852 - val_accuracy: 0.8890
Epoch 6/10
16/16 [==============================] - 9s 548ms/step - loss: 0.3441 - accuracy: 0.8650 - val_loss: 0.2691 - val_accuracy: 0.9010
Epoch 7/10
16/16 [==============================] - 9s 54

## Evaluating the Transfer Learning Model

In [17]:
validation_loss, validation_accuracy = model.evaluate_generator(generator=validation_generator)
print("Transfer Learning Accuracy is ", validation_accuracy)

Instructions for updating:
Please use Model.evaluate, which supports generators.
  ...
    to  
  ['...']
Transfer Learning Accuracy is  0.911


# Fine Tuning the Model Further

Fine tune only the top(output-ish) few layers to adapt to the custom domain: Because the lower(input-ish) layers will probably extract features like line edges, color differences etc which are common across all animals and the upper(output-ish) layers will extract the more specific features like eyes, nose, ears etc which are specidic to cats and dogs. To adapt to a domain, it is important to make the model adapt to these specific cases of dog ears and cat ears which are found near the output as opposed to something which is generic for both dogs and cats line edges and color gradients found near the inputs.

In short, fine tune the outputtish layers. One common approach is to only fine tune the output dense and softmax layers to convert the 10 animals dataset to cats and dogs.

Fine tune only after transfer learning

In [21]:
print("Total number of layers in the model is ", len(base_model.layers))

Total number of layers in the model is  155


In [0]:
# Unfreeze some of the top(output-ish) layers of the base model
# For this, first unfreeze the whole model
base_model.trainable = True

# Let's fine tune after the 100th layer onwards
fine_tune_at = 100

# Now freeze all layers near the input till 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Now out of the 150 layers, the the last(output-ish) 55 layers can be fine tuned

## Compile the Fine Tuned Model

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Training(Fine Tuning) the Fine Tuning Model

In [28]:
model.fit_generator(generator=train_generator, 
                    epochs=5, 
                    validation_data=validation_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/5
16/16 [==============================] - 15s 949ms/step - loss: 0.1466 - accuracy: 0.9470 - val_loss: 0.0843 - val_accuracy: 0.9660
Epoch 2/5
16/16 [==============================] - 9s 577ms/step - loss: 0.0147 - accuracy: 0.9995 - val_loss: 0.0836 - val_accuracy: 0.9710
Epoch 3/5
16/16 [==============================] - 9s 574ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0924 - val_accuracy: 0.9690
Epoch 4/5
16/16 [==============================] - 9s 570ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.1045 - val_accuracy: 0.9670
Epoch 5/5
16/16 [==============================] - 9s 572ms/step - loss: 8.4406e-04 - accuracy: 1.0000 - val_loss: 0.1145 - val_accuracy: 0.9660


## Evaluating the Fine Tuned Model

In [29]:
validation_loss_fine_tuned, validation_accuracy_fine_tuned = model.evaluate_generator(
    generator=validation_generator
)

  ...
    to  
  ['...']


In [33]:
print("Validation accuracy after fine tuning is ", validation_accuracy_fine_tuned)

print("Model improved")

Validation accuracy after fine tuning is  0.966
Model improved
